In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
sys.path.append("../")

In [3]:
import pickle
import numpy as np
import pandas as pd
from tqdm import tqdm_notebook as tqdm
import jieba

In [4]:
import textsim

In [5]:
comments_path = textsim.get_resource_path("toxicity-data-merged-comments-05251530.csv")
comments = pd.read_csv(comments_path, index_col=0)
posts_path = textsim.get_resource_path("toxicity-data-merged-posts-05251530.csv")
posts = pd.read_csv(posts_path, index_col=0)

## remove all posts does not have comments

In [6]:
posts.head()

,TextContent,TextDate,TextSrc,TextTitle,TextUrl,Toxicity
TextId,,,,,,
Jessy001,蔡英文總統今天參訪位於台中港二號碼頭的「海洋示範風場」，稍早她透過臉書提問「你們知道風機葉片...,20190523,udn,風機葉片就是證據 蔡英文：民進黨比國民黨更會建設\n,https://udn.com/news/story/6656/3830874,NaN
Jessy002,昨（20）日是蔡英文2016年上任以來，執政期滿3年，明年台灣將舉行總統大選，受到17日立法...,20190521,udn,綠委助陣 一張圖全解「蔡英文執政三年做了啥」,https://udn.com/news/story/120489/3825878,NaN
Jessy003,行政院前院長賴清德日前提及他跟蔡英文總統的三個不同，蔡英文昨天接受廣播節目「POP大國民」專...,20190522,udn,蔡英文：我做總統比較有經驗,https://udn.com/news/story/11311/3826839,NaN
Jessy004,對於總統蔡英文就職三周年的演說，主打MIT upgrade，並說兩岸外交很好，郭台銘反問媒體...,20190520,udn,蔡英文就職3周年 郭台銘：就是她「做衰」台灣,https://udn.com/news/story/6656/3823470,NaN
Jessy005,《ETtoday新聞雲》昨公布「蔡英文施政滿意度調查」結果，對於蔡總統整體施政滿意度，34....,20190524,udn,蔡英文施政滿意度34.9％ 減稅58.3％最有感,https://udn.com/news/story/6656/3831417,NaN


In [7]:
comments_textids = set(comments.TextId.tolist())
posts["TextId"] = posts.index
posts = posts.loc[posts.TextId.apply(lambda x: x in comments_textids), :]

In [8]:
posts_ids = posts.TextId.tolist()
posts_texts = posts.TextContent

## Data preparation

In [9]:
posts_comments = {}
for k, g in comments.groupby('TextId'):
    posts_comments[k] = [x.replace("\\", "") for x in g.CommentContent.tolist()]

posts_comments_path = textsim.get_data_path("text_materials.pkl")
with open(posts_comments_path, "wb") as fout:
    pickle.dump((posts_ids, posts_comments), fout)

In [10]:
next(iter(posts_comments.items()))

('Jessy001', ['花大錢，當凱子的證據嗎？', '我只看到菜皇比國民黨更會灑大錢~'])

In [11]:
posts_tokens = [" ".join(jieba.lcut(x)) for x in posts_texts]

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\seantyh\AppData\Local\Temp\jieba.cache
Loading model cost 0.646 seconds.
Prefix dict has been built succesfully.


In [12]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [13]:
def identity_func(x):
    return x
tfidf = TfidfVectorizer(min_df=3, max_df=0.8)
M = tfidf.fit_transform(posts_tokens)
tfidf_path = textsim.get_data_path("tfidf.pkl")
with open(tfidf_path, "wb") as fout:
    pickle.dump(tfidf, fout)

In [14]:
M.shape

(51, 575)

In [15]:
from scipy.sparse.linalg import svds

In [16]:
u, s, vt = svds(M, k=10)
u_norm = u / np.expand_dims(np.sqrt(np.diag(np.dot(u, u.transpose()))), 1)
svd_path = textsim.get_data_path("svd.pkl")
with open(svd_path, "wb") as fout:
    pickle.dump((u_norm, s, vt), fout)

## Test SimilarComments object

In [17]:
from textsim import SimilarComments

In [18]:
intext = """「政黨存在的價值，就是透過選舉贏得政權。」若從這個普遍被接受的邏輯出發，民進黨初選的「英德內戰」，完全可以理解，唯一的「誤會」是，他們認定只有自己才可以為所屬政黨贏得政權的推論基礎，未必是事實；同樣的，高雄市長韓國瑜才當選就卯足全勁拚總統大選，立論於只有他可以為國民黨贏得政權，亦未必是事實，還有一舉雙殺總統與高雄市長的風險。
"""

In [19]:
intext2 = """高雄市長韓國瑜首場總統初選造勢「決戰2020．贏回台灣」，將在6月1日總統府前凱達格蘭大道舉行，讓各地韓粉相當興奮。國民黨籍台北市議員王鴻薇認為，韓國瑜現在可能面臨要選2020前的最低潮時刻，若是能撐過這波谷底的低潮，就真的會是一員強將了。

韓國瑜鬆口願角逐2020後，近期就傳出中部韓粉準備在6月22日在台中舉辦挺韓大會；在此之前，因韓流快速擴張，首場韓國瑜初選大造勢已敲定6月1日登場，且定名為「決戰2020．贏回台灣」。

據中評社報導，王鴻薇提及，韓國瑜之前在市議會質詢、表態參選2020，遭民眾質疑誠信與正當性，加上網路一片黑函情況下，韓好像做什麼都錯，跟選前差別非常大。她指出，網路上韓國瑜的消息十之八九都是負面的，原因就在2018選前只有民進黨的攻擊，但在黨內初選階段，則多了黨內派系的攻擊。
"""

In [20]:
sim_comm = SimilarComments()

In [21]:
sim_comm.comments(intext)

['2020剿滅民進黨這個土匪窩和空心菜這隻妖孽',
 '而蔡慰安婦才是真正讓台灣政治經濟衰敗的罪魁禍首。',
 '有什麼經驗?我們看到的是有做得很爛的徑驗,所以要把她下架. ',
 '她看起來變成一個充滿意識形態又頑固的女人,又有點無知和鴕鳥的心態.畢竟有的人老了就會變成這樣的人. ',
 '蔡英文實在不配稱民主!',
 '竟敢自比做總統最有經驗？ \r\n除了二蔣外目前就是李登輝最有經驗了，要不要請最有當總統經驗的李登輝回來當總統？',
 '你又崩潰了？ \r\n笑死人！你們無恥綠膿包怎麼永遠的兩套標準？ \r\n^_^ ',
 '最沒信用的破麻有什麼好講的,明年讓這破麻下台最重要. ',
 '蔡英文是一個極權女人，只愛權力，吃勞工青年人豆腐。']

In [22]:
sim_comm.comments(intext2)

['英皇震怒 : 怎麼可能 我應該比姚文智92%還要強才對啊',
 '最沒信用的破麻有什麼好講的,明年讓這破麻下台最重要. ',
 '皇民文化官員把訊息告訴日本算不算犯罪?比照綠黨對大陸, 高官卸任請六年不得訪日. 日本侵占台灣海權魚權....',
 '聲望剩下18%還可以趴趴走，厲害了蔡女皇',
 '她看起來變成一個充滿意識形態又頑固的女人,又有點無知和鴕鳥的心態.畢竟有的人老了就會變成這樣的人. ',
 '而蔡慰安婦才是真正讓台灣政治經濟衰敗的罪魁禍首。',
 '是呀，因為真的跟一般人理解剛好相反，她的腦袋長在屁股上……',
 '你又崩潰了？ \r\n笑死人！你們無恥綠膿包怎麼永遠的兩套標準？ \r\n^_^ ',
 '東廠各廠公開會，商議如何陷害忠良，把台灣出賣給美國日本，讓台灣人世世代代為奴。',
 '繼續挑釁中共，等待事發，宣佈戒嚴，總統任期無限']